In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import copy
import networkx as nx
import random
from random import sample

#### Load data

In [2]:
FlightsNx = pd.read_pickle("FlightsNx18.pkl")

#### Implement as network. 


In [3]:
Network = nx.from_pandas_edgelist(FlightsNx, source = "Origin", target = "Dest", edge_attr = True)
print(nx.info(Network))

Name: 
Type: Graph
Number of nodes: 358
Number of edges: 3220
Average degree:  17.9888


In [4]:
degree_dict = nx.algorithms.centrality.degree_centrality(Network) 
degree_values = list(degree_dict.values())
# Create dataframe
degrees = pd.DataFrame(degree_values)
degrees.columns = ["degree"]
degrees["degree_unnormalized"] = degrees["degree"]*(len(Network.nodes)-1)
degrees["airport"] = degree_dict.keys()

## Attacks and Errors
### Analysis of how removing certain airports affect network characteristics

#### Measures for entire network

In [ ]:
print(nx.info(Network))
print("The average shortest path is: {}".format(nx.average_shortest_path_length(Network)))
print("The average clustering coefficient is: {}".format(nx.average_clustering(Network)))
print("The diameter of the network is: {}".format(nx.diameter(Network)))
print("The global efficiency of the network is: {}".format(nx.global_efficiency(Network)))

Name: 
Type: Graph
Number of nodes: 358
Number of edges: 3220
Average degree:  17.9888
The average shortest path is: 2.3794657527815595
The average clustering coefficient is: 0.5671628544291067
The diameter of the network is: 6
The global efficiency of the network is: 0.4571454652628847


In [ ]:
# What happens to network characteristics as the most important airports are removed? 
degrees.sort_values(by = "degree_unnormalized", inplace = True, ascending = False)
average_clustering = list()
average_shortest_path_length = list()
global_efficiency = list()
average_degree = list()
for i in range(30):
    net = Network.subgraph(nodes = list(degrees["airport"])[i:])
    average_clustering.append(nx.average_clustering(net))
    
    #average_shortest_path_length.append(nx.average_shortest_path_length(net))
    
    global_efficiency.append(nx.global_efficiency(net))
    
    average_degree.append(sum(dict(net.degree()).values())/len(dict(net.degree()).values()))
    if i%5 == 0:
        print(i)
    
measures_attacks = pd.DataFrame({"average_degree":average_degree,
                                "average_clustering":average_clustering,
                                "global_efficiency":global_efficiency})
#measures_attacks.head()

0


In [ ]:
measures_attacks.to_pickle("measures_attacks.pkl")

In [ ]:
## What happens to network characteristics if nodes are randomly removed? 
random.seed(a = 12345)
degrees.sort_values(by = "degree_unnormalized", inplace = True, ascending = False)
average_clustering = list()
#average_shortest_path_length = list()
#global_efficiency = list()
average_degree = list()
for i in range(300):
    net = Network.subgraph(nodes = sample(list(degrees["airport"]), len(list(degrees["airport"]))-i))
    average_clustering.append(nx.average_clustering(net))
    
    #average_shortest_path_length.append(nx.average_shortest_path_length(net))
    
    #global_efficiency.append(nx.global_efficiency(net))
    
    average_degree.append(sum(dict(net.degree()).values())/len(dict(net.degree()).values()))
    if i%5 == 0:
        print(i)
    
measures_random = pd.DataFrame({"average_degree":average_degree,
                                "average_clustering":average_clustering})
measures_random.head()

In [ ]:
measures_random.to_pickle("measures_random.pkl")

In [ ]:
# What happens to network characteristics as the least important nodes are removed?
degrees.sort_values(by = "degree_unnormalized", inplace = True, ascending = False)
average_clustering = list()
average_shortest_path_length = list()
global_efficiency = list()
average_degree = list()
for i in range(30):
    net = Network.subgraph(nodes = list(degrees["airport"])[:len(degrees["airport"])-i])
    average_clustering.append(nx.average_clustering(net))
    
    #average_shortest_path_length.append(nx.average_shortest_path_length(net))
    
    global_efficiency.append(nx.global_efficiency(net))
    
    average_degree.append(sum(dict(net.degree()).values())/len(dict(net.degree()).values()))
    if i%5 == 0:
        print(i)
    
measures_errors = pd.DataFrame({"average_degree":average_degree,
                                "average_clustering":average_clustering,
                                "global_efficiency":global_efficiency})
measures_errors.head()

In [ ]:
measures_errors.to_pickle("measures_errors.pkl")

#### Mapping airport network geographically

In [ ]:
attack_effect = pd.read_pickle("measures_attacks.pkl")
error_effect = pd.read_pickle("measures_errors.pkl")
random_effect = pd.read_pickle("measures_random.pkl")
plist = list(range(1,31))
plist = [(x-1) * 1/310 for x in plist]
plist2 = list(range(1,301))
plist2 = [(x-1) * 1/310 for x in plist2]

In [ ]:
len(plist)

In [ ]:
fig, ax = plt.subplots(figsize = (18,6))
plt.plot(plist2, random_effect["average_degree"], label = "Random", marker = "x")
#plt.title("Effect of attacks and errors on network average degree")
plt.legend(loc = "lower left")
plt.xlabel("Fraction of airports removed")
plt.ylabel("Average degree")
plt.ylim(0,20)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (18,6))
plt.subplot(1,2,1)
plt.plot(plist, attack_effect["average_degree"], label = "Attacks", marker = "*")
plt.plot(plist, error_effect["average_degree"], label = "Errors", marker = "o")
plt.plot(plist, random_effect["average_degree"][0:30], label = "Random", marker = "x")
#plt.title("Effect of attacks and errors on network average degree")
plt.legend(loc = "lower left")
plt.xlabel("Fraction of airports removed")
plt.ylabel("Average degree")

plt.subplot(1,2,2)
plt.plot(plist, attack_effect["average_clustering"], label = "Attacks", marker = "*")
plt.plot(plist, error_effect["average_clustering"], label = "Errors", marker = "o")
plt.plot(plist, random_effect["average_clustering"][0:30], label = "Random", marker = "x")
#plt.title("Effect of attacks and errors on network clustering coefficient")
plt.legend(loc = "lower left")
plt.xlabel("Fraction of airports removed")
plt.ylabel("Clustering coefficient")

plt.savefig(fname = "Figures/attacksanderrors.png", format = "png", bbox_inches = "tight")

plt.show()